In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import re

# 1.Load sqlite database

In [2]:
def load_dataset(db, table):
    """
    db: str, sqlite3 databes file name
    table: str, sqlite table name
    """
    dataset = pd.read_sql(f"SELECT * FROM {table}", sql.connect(db))
    dataset['file_size'] = dataset['file_size'] / (1024 * 1024)
    dataset['file_size'] =  dataset['file_size'].astype(int)
    return dataset

In [3]:
dataset = load_dataset('database.db', 'movies_data')
dataset.sample(4)

,message_id,file_id,caption,file_size,id
400,949,BAACAgIAAx0Ec8rVjQACA7VkYdhOQAEZcU0wKk5xnK7l1Z...,PREMYERA⚡️ 2023\n\n🎥NOMI Havfli hudud | 6 QISM...,143,401
138,687,BAACAgIAAx0Ec8rVjQACAq9kYdLmiWtAhPXrmaGgkUMabD...,Jumong Afsonasi\n\n77 - 78 - qismlar\nTili: oʻ...,574,139
1645,2194,BAACAgQAAx0Ec8rVjQACCJJkYfpNP3jtnsb2khjLJjcFTf...,Vorislar 13-qism\n\nTili: oʻzbek tilida\nJan...,370,1646
1221,1770,BAACAgEAAx0Ec8rVjQACBupkYfFA1Rk2JJ2if3gYhFnkFA...,🎬 ➺ Asteriks va Sehrli damlama \n🇺🇿 ➺ O'zbek T...,857,1222


# 2.Colect title

In [4]:
def clean_title(dataset_title):
    respond = []
    for caption in dataset_title.str.split('\n'):
        n = len(caption)
        if re.search('premyera', caption[0].lower()) and n >= 2:
            if caption[1].strip() == '' and n >= 3:
                respond.append(caption[2])
            else:
                respond.append(caption[1]) 
        else:
            if len(caption[0]) >= 50:
                respond.append(caption[0][0:49])
            else:
                respond.append(caption[0])
    return respond

In [5]:
dataset['title'] = clean_title(dataset.caption)

In [21]:
stop_words = [
              r"19[0-9][0-9]|20[0-2][0-9]",
              r"o'zbek film|Uzbek tilida|Hind kino",
              r"O'zbek tilida|Uzbek kino|o'zbek kino|o'zbek кино",
              r"Nomi.*:|Kino nomi:|NOMI|Davlat :",
              r"🖥|🌎|🚑|🩺|🎞|📺|🎬|🎥|b/f|#|'|HD|»|«|▷|🤡|➺",
#               r"Ozbek Tilida|[SD]|【|】",
              r"[.*]|✎|Kino:|Film noмi:|\[\]|📹|✅|🔥|❓",
              r"[0-9][0-9][0-9]p|Multserial:|☃️|❄️|\(|\)",
              r"🔸|🔶|🍿|🔰|\[\]|✏️|-|Nomi|🇺🇿 📆 Yili 📁 Formati",
              r"Oʻzbek tilida|💎|.remyera|PREMYERA|𝐦𝐟|𝐦/𝐟 |m/s",
              r"➣|Ozbek.*$|🇺🇿|RetroSerial|RetroMultserial|Uzbek.*$",
              r":|B/F|,"]


In [22]:
def clear_title2(dataset_title, matches):
    for match in matches:
        dataset_title = dataset_title.str.replace(match, '', regex = True)
    return dataset_title

In [23]:
dataset['title'] = clear_title2(dataset.title, stop_words)

In [24]:
state = 0
if state == 1:
    for title in dataset['title']:
        print("------")
        print(title)
else:
    for title in clear_title2(dataset['title'], stop_words):
        print("------")
        print(title)

------
Ozga sayyoralik mahbubim 6qism Koreya seriali
------
Ozga sayyoralik mahbubim 7qism Koreya seriali 
------
Ozga sayyoralik mahbubim 8qism Koreya seriali
------
Ozga sayyoralik mahbubim 9qism Koreya seriali
------
Ozga sayyoralik mahbubim 10qism Koreya seriali 
------
Ozga sayyoralik mahbubim 11qism Koreya seriali
------
Ozga sayyoralik mahbubim 12qism Koreya seriali 
------
Ozga sayyoralik mahbubim 13qism Koreya seriali 
------
Ozga sayyoralik mahbubim 14qism Koreya seriali 
------
Ozga sayyoralik mahbubim 15qism Koreya seriali 
------
Ozga sayyoralik mahbubim 16qism Koreya seriali 
------
Ozga sayyoralik mahbubim 17qism Koreya seriali 
------
Ozga sayyoralik mahbubim 18qism Koreya seriali 
------
Ozga sayyoralik mahbubim 19qism Koreya seriali 
------
Ozga sayyoralik mahbubim 20qism Koreya seriali 
------
Ozga sayyoralik mahbubim 21qism Koreya seriali 
------
Ozga sayyoralik mahbubim 22qism Koreya seriali 
------
Ozga sayyoralik mahbubim 23qism Koreya seriali 
------
Ozga sayyor

# 3. Clean Caption

In [10]:
def clean_caption(dataset_caption, matches):
    for match in matches:
        dataset_caption = dataset_caption.str.replace(match, '', regex=True)
    return dataset_caption

In [27]:
matches = [
          r'@.* ',
          r'@.*\n',
          r'@Jahon_Primyeralar',
          r'@Tarjima_Kinolar_bbot',
          r"https://telegram.me/+(.*)([a-z])",
          r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]",
          r'@KINOLAR_VA_SERIALLAR_22',
          r'👉 i',
          r'Diniy savollar',
          r'🔍 Kino topib beruvchi bot:',
          r'Kanalga ulanish👇👇👇',
          r'🌐Kanalimiz: @TaronaMove',
          r'🔍Musiqa Bot: 🎧',
          r"🌐 Kanalimiz:",
          r'👉 👈',
          r'👇👉',
          r'👇|,'
          ]

In [28]:
dataset['caption'] = clean_caption(dataset.caption, matches)
for caption in dataset['caption']:
    print("-----")
    print(caption)

-----
O'zga sayyoralik mahbubim 6-qism Koreya seriali

 
-----
O'zga sayyoralik mahbubim 7-qism Koreya seriali 

 
-----
O'zga sayyoralik mahbubim 8-qism Koreya seriali

 
-----
O'zga sayyoralik mahbubim 9-qism Koreya seriali

 
-----
O'zga sayyoralik mahbubim 10-qism Koreya seriali 

 
-----
O'zga sayyoralik mahbubim 11-qism Koreya seriali

 
-----
O'zga sayyoralik mahbubim 12-qism Koreya seriali 

 
-----
O'zga sayyoralik mahbubim 13-qism Koreya seriali 

 
-----
O'zga sayyoralik mahbubim 14-qism Koreya seriali 

 
-----
O'zga sayyoralik mahbubim 15-qism Koreya seriali 

 
-----
O'zga sayyoralik mahbubim 16-qism Koreya seriali 

 
-----
O'zga sayyoralik mahbubim 17-qism Koreya seriali 

 
-----
O'zga sayyoralik mahbubim 18-qism Koreya seriali 

 
-----
O'zga sayyoralik mahbubim 19-qism Koreya seriali 

 
-----
O'zga sayyoralik mahbubim 20-qism Koreya seriali 

 
-----
O'zga sayyoralik mahbubim 21-qism Koreya seriali 

 
-----
O'zga sayyoralik mahbubim 22-qism Koreya seriali 

 
-----

# 4.Seperate Series

In [29]:
def movi_or_series(dataset_caption):
    n = 0
    series = []
    movies = []

    for caption in dataset['caption']:
        if re.search(r"serial|G‘alati narsalar|Jumong|SHUNQOR|Quyosh avlodlari", caption):
            n+=1
            print("--------")
            print(caption)
            series.append(True)
            movies.append(False)
        elif re.search(r"Vorislar|Soʻl Yonim|Sirli.Olam|seriya|SHIFOKOR QISMATI", caption): #sayyoralik mahbubim 
            n+=1
            print("--------")
            print(caption)
            series.append(True)
            movies.append(False)
        elif re.search(r"sayyoralik mahbubim", caption):
            n+=1
            print("--------")
            print(caption)
            series.append(True)
            movies.append(False)
        else:
            series.append(False)
            movies.append(True)
    return series, movies, n

In [30]:
series, movies, n = movi_or_series(dataset.caption)

--------
O'zga sayyoralik mahbubim 6-qism Koreya seriali

 
--------
O'zga sayyoralik mahbubim 7-qism Koreya seriali 

 
--------
O'zga sayyoralik mahbubim 8-qism Koreya seriali

 
--------
O'zga sayyoralik mahbubim 9-qism Koreya seriali

 
--------
O'zga sayyoralik mahbubim 10-qism Koreya seriali 

 
--------
O'zga sayyoralik mahbubim 11-qism Koreya seriali

 
--------
O'zga sayyoralik mahbubim 12-qism Koreya seriali 

 
--------
O'zga sayyoralik mahbubim 13-qism Koreya seriali 

 
--------
O'zga sayyoralik mahbubim 14-qism Koreya seriali 

 
--------
O'zga sayyoralik mahbubim 15-qism Koreya seriali 

 
--------
O'zga sayyoralik mahbubim 16-qism Koreya seriali 

 
--------
O'zga sayyoralik mahbubim 17-qism Koreya seriali 

 
--------
O'zga sayyoralik mahbubim 18-qism Koreya seriali 

 
--------
O'zga sayyoralik mahbubim 19-qism Koreya seriali 

 
--------
O'zga sayyoralik mahbubim 20-qism Koreya seriali 

 
--------
O'zga sayyoralik mahbubim 21-qism Koreya seriali 

 
--------
O'zga s

In [31]:
n

306

In [32]:
series_dataset = dataset[series]
series_dataset.reset_index()

,index,message_id,file_id,caption,file_size,id,title
0,0,549,BAACAgQAAx0Ec8rVjQACAiVkYdAEXQkqHLqbBcqIH3vNKO...,O'zga sayyoralik mahbubim 6-qism Koreya serial...,237,1,Ozga sayyoralik mahbubim 6qism Koreya seriali
1,1,550,BAACAgQAAx0Ec8rVjQACAiZkYdAEM3169YIvOG_saKba8O...,O'zga sayyoralik mahbubim 7-qism Koreya serial...,219,2,Ozga sayyoralik mahbubim 7qism Koreya seriali
2,2,551,BAACAgQAAx0Ec8rVjQACAidkYdAEof5Ay_o6DWJ94Btx4D...,O'zga sayyoralik mahbubim 8-qism Koreya serial...,195,3,Ozga sayyoralik mahbubim 8qism Koreya seriali
3,3,552,BAACAgQAAx0Ec8rVjQACAihkYdAE5R4NV4IWkjgL2RrK3D...,O'zga sayyoralik mahbubim 9-qism Koreya serial...,181,4,Ozga sayyoralik mahbubim 9qism Koreya seriali
4,4,553,BAACAgQAAx0Ec8rVjQACAilkYdAE7Avw8KwdQGDjhDsgZ9...,O'zga sayyoralik mahbubim 10-qism Koreya seria...,183,5,Ozga sayyoralik mahbubim 10qism Koreya seriali
...,...,...,...,...,...,...,...
301,1648,2197,BAACAgQAAx0Ec8rVjQACCJVkYfq_deBMIQ3V-dFLaHxO4H...,Vorislar 16-qism\n\nTili: oʻzbek tilida\nJan...,344,1649,Vorislar 16qism
302,1649,2198,BAACAgQAAx0Ec8rVjQACCJZkYfq_Jzgu1LT2meHgMuOFfP...,Vorislar 17-qism\n\nTili: oʻzbek tilida\nJan...,341,1650,Vorislar 17qism
303,1650,2199,BAACAgQAAx0Ec8rVjQACCJdkYfrAklY-VepxqP4lY7CCNG...,Vorislar 18-qism\n\nTili: oʻzbek tilida\nJan...,342,1651,Vorislar 18qism
304,1651,2200,BAACAgQAAx0Ec8rVjQACCJhkYfrE1EzqJlPL46JrhYyKcp...,Vorislar 19-qism\n\nTili: oʻzbek tilida\nJan...,374,1652,Vorislar 19qism


In [33]:
dataset_movies = dataset[movies]
dataset_movies.reset_index()

,index,message_id,file_id,caption,file_size,id,title
0,26,575,BAACAgEAAx0Ec8rVjQACAj9kYdCEAAEKRCI6ctDbfmVByT...,Zorro\n\nTili: oʻzbek tilida\nJanri: jangari s...,819,27,Zorro
1,27,576,BAACAgQAAx0Ec8rVjQACAkBkYdCE35Gw-QajJXdcW6T9uQ...,Qonli tatil\n\nTili: oʻzbek tilida\nJanri: kom...,1009,28,Qonli tatil
2,28,577,BAACAgEAAx0Ec8rVjQACAkFkYdCL7HJSK8hHvlkRyBhS29...,Qo'rqmas\n\nTili: O'zbek tilida\nSifati 480p\...,671,29,Qorqmas
3,29,578,BAACAgIAAx0Ec8rVjQACAkJkYdCLHiSOf6eWgDEoetKgNT...,#tarjima #romantik #melodramma\n\nNomi: Shamol...,539,30,tarjima romantik melodramma
4,30,579,BAACAgQAAx0Ec8rVjQACAkNkYdCLpDPo0pBUaMjV_k8QZY...,Nomi: Qonli O'yin yohud qimirlagan O'ladi\n\nD...,370,31,Qonli Oyin yohud qimirlagan Oladi
...,...,...,...,...,...,...,...
1357,1663,2212,BAACAgQAAx0Ec8rVjQACCKRkYfsTf9j1Qa039fmXAn4xJS...,Jangchilar darvozasi\n\nDavlati: Fransiya Xito...,928,1664,Jangchilar darvozasi
1358,1664,2213,BAACAgUAAx0Ec8rVjQACCKVkYfsT2y9fzaK0qyVgAAHTUX...,Jangchilar darvozasi\n\nDavlati: Fransiya Xito...,2190,1665,Jangchilar darvozasi
1359,1665,2214,BAACAgQAAx0Ec8rVjQACCKZkYfsUGwgFl3e3KH0RcQuAhZ...,🔥#Premyera\n🎬Nomi: Troll\n➖➖➖➖➖➖➖➖➖➖\n🇺🇿Tili: ...,686,1666,Troll
1360,1666,2215,BAACAgIAAx0Ec8rVjQACCKdkYfsU8pZMTaDrbSiqvE6XWZ...,🔥#Premyera\n🎬Nomi: Etik Kiygan Mushuk 2: So'ng...,846,1667,Songi Tilak


In [34]:
for n in dataset_movies.title:
    print("------")
    print(n)

------
Zorro
------
Qonli tatil
------
Qorqmas
------
tarjima romantik melodramma
------
 Qonli Oyin yohud qimirlagan Oladi
------
Yulduzlar Aybdormi
------
Omad
------
Yovuz oʻliklar 2
------
Yovuz oʻliklar 2
------
Shoqol 
------
Shoqol 
------
 Chupa
------
 Chupa
------
 Chupa
------
Имя 
------
Morgdagi qotillik  
------
Morgdagi qotillik  
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Daxshatli Uy
------
Daxshatli Uy
------
Daxshatli Uy
------
Daxshatli Uy
------
 Daxshatli UY
------
 6qism
------
 7qism
------
 8qism
------
 Daxshatli UY
------
 Daxshatli UY
------
 OSVENSIM CHEMPIONI 
------
Volfxaund / Wolf Hound
------
Zombilar makoni
--

In [35]:
dataset_movies.loc[:,'title'] = dataset_movies['title'].astype(str).str.strip()

In [36]:
dataset_movies.to_csv('dataset_movies.csv', index = False)
series_dataset.to_csv('series_dataset.csv', index = False)